# Audiobooks business case

## Preprocess the data. Balance the dataset. Create 3 datasets: training, validation, and test. Save the newly created sets in a tensor friendly format (e.g. *.npz)

Since we are dealing with real life data, we will need to preprocess it a bit before training the model.


### Extract the data from the csv

In [15]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing


raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter = ',')

unscaled_inputs_all = raw_csv_data[:,1:-1]

targets_all = raw_csv_data[:,-1]

### Balance the dataset

In [2]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] ==0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0)
targets_equal_priors = np.delete (targets_all, indices_to_remove, axis=0)

### Standardize the inputs

In [3]:
scaler = StandardScaler()
# scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)
scaled_inputs = scaler.fit_transform(unscaled_inputs_equal_priors)

### Shuffle the data

In [4]:

shuffled_indices = np.arange(scaled_inputs.shape[0])

np.random.shuffle(shuffled_indices)
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]




### Split the dataset into train, validation, and test

In [5]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1754.0 3579 0.49008102822017324
234.0 447 0.5234899328859061
249.0 448 0.5558035714285714


### Save the three datasets in *.npz

In [6]:
# np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
# np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
# np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

In [7]:
# npz = np.load('Audiobooks_data_train.npz')
# train_inputs = npz['inputs'].astype(np.float32)
# tarin_targets = npz['targets'].astype(np.int64)

# npz = np.load('Audiobooks_data_validation.npz')
# validation_inputs = npz['inputs'].astype(np.float32)
# validation_targets= npz['targets'].astype(np.int64)

# npz= np.load('Audiobooks_data_test.npz')
# test_inputs = npz['inputs'].astype(np.float32)
# test_targets=npz['targets'].astype(np.int64)

## Learning

In [8]:
# input_size=10
output_size=2
hidden_layer_size=100

model = tf.keras.Sequential([
                            
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax'),
    tf.keras.layers.Dense(output_size, activation='softmax'),
    tf.keras.layers.Dense(output_size, activation='softmax')
    
                            ])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size=100
max_epochs = 10
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit( train_inputs,
         train_targets,
         batch_size=batch_size,
         epochs=max_epochs,
          callbacks = [early_stopping],
         validation_data = (validation_inputs, validation_targets),
         verbose=2)

Epoch 1/10
36/36 - 1s - loss: 0.6937 - accuracy: 0.5099 - val_loss: 0.6956 - val_accuracy: 0.4765 - 1s/epoch - 36ms/step
Epoch 2/10
36/36 - 0s - loss: 0.6904 - accuracy: 0.5099 - val_loss: 0.6909 - val_accuracy: 0.4765 - 79ms/epoch - 2ms/step
Epoch 3/10
36/36 - 0s - loss: 0.6861 - accuracy: 0.5099 - val_loss: 0.6844 - val_accuracy: 0.4765 - 78ms/epoch - 2ms/step
Epoch 4/10
36/36 - 0s - loss: 0.6788 - accuracy: 0.5099 - val_loss: 0.6744 - val_accuracy: 0.7830 - 86ms/epoch - 2ms/step
Epoch 5/10
36/36 - 0s - loss: 0.6671 - accuracy: 0.8997 - val_loss: 0.6601 - val_accuracy: 0.9172 - 85ms/epoch - 2ms/step
Epoch 6/10
36/36 - 0s - loss: 0.6511 - accuracy: 0.9058 - val_loss: 0.6405 - val_accuracy: 0.9217 - 110ms/epoch - 3ms/step
Epoch 7/10
36/36 - 0s - loss: 0.6311 - accuracy: 0.9067 - val_loss: 0.6196 - val_accuracy: 0.9105 - 105ms/epoch - 3ms/step
Epoch 8/10
36/36 - 0s - loss: 0.6083 - accuracy: 0.9075 - val_loss: 0.5946 - val_accuracy: 0.9150 - 91ms/epoch - 3ms/step
Epoch 9/10
36/36 - 0s -

## Testing the model

In [9]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.5566 - accuracy: 0.8884


In [10]:
print('\nTest loss: {0:.4f}. Test accuracy: {1:.4f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.5566. Test accuracy: 88.8393%


In [11]:
model.predict(test_inputs[:10])

array([[0.40696865, 0.59303135],
       [0.4069686 , 0.5930314 ],
       [0.40697646, 0.59302354],
       [0.4069686 , 0.5930314 ],
       [0.4069686 , 0.5930314 ],
       [0.60852164, 0.3914784 ],
       [0.60852164, 0.3914784 ],
       [0.40696865, 0.59303135],
       [0.60852164, 0.3914784 ],
       [0.40697107, 0.59302896]], dtype=float32)

In [12]:
df=pd.read_csv("Audiobooks_data.csv")

## Predicting

In [17]:
new_inputs = [[1620, 810, 5.87, 5.87, 0, 5, 0, 1, 0, 253]]
scaled = scaler.fit_transform(new_inputs)

num = np.argmax(model.predict(scaled), axis=-1)
print(num)   

[0]
